In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest

In [2]:
path = 'D:/Study/McGill University - MMA/Courses/INSY 695/Team Project/Data/'
df_raw = pd.read_csv(path+"media prediction and its cost NEW.csv")

In [3]:
num_cols, cate_cols, pre_dum_cols = [], [], []
for c in df_raw.columns:
    uni_len = len(set(df_raw[c]))
    dtype = df_raw[c].dtypes

    if dtype == object:
        cate_cols.append(c)
    else: # dtype == float
        if uni_len == 2:
            pre_dum_cols.append(c)
        elif uni_len < 10:
            num_cols.append(c)
            print("Number of unique values: %d \t Datatype: %s \t Column name: %s" %(uni_len, dtype, c))
        else:
            num_cols.append(c)

Number of unique values: 6 	 Datatype: int64 	 Column name: unit_sales(in millions)
Number of unique values: 6 	 Datatype: int64 	 Column name: total_children
Number of unique values: 5 	 Datatype: int64 	 Column name: avg_cars_at home(approx)
Number of unique values: 6 	 Datatype: int64 	 Column name: num_children_at_home
Number of unique values: 5 	 Datatype: int64 	 Column name: avg_cars_at home(approx).1


In [4]:
df_raw.head()

,food_category,food_department,food_family,store_sales(in millions),store_cost(in millions),unit_sales(in millions),promotion_name,sales_country,marital_status,gender,...,grocery_sqft,frozen_sqft,meat_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,media_type,cost
0,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4,Bag Stuffers,USA,M,F,...,8670,5415,13610,1,1,1,1,1,"Daily Paper, Radio",126.62
1,Breakfast Foods,Frozen Foods,Food,7.36,2.7232,4,Bag Stuffers,USA,M,F,...,18670,5415,3610,1,1,1,1,1,"Daily Paper, Radio",126.62
2,Breakfast Foods,Frozen Foods,Food,5.52,2.5944,3,Cash Register Lottery,USA,M,M,...,18670,5415,3610,1,1,1,1,1,"Daily Paper, Radio",59.86
3,Breakfast Foods,Frozen Foods,Food,3.68,1.3616,2,High Roller Savings,USA,S,F,...,18670,5415,3610,1,1,1,1,1,"Daily Paper, Radio",84.16
4,Breakfast Foods,Frozen Foods,Food,3.68,1.1776,2,Cash Register Lottery,USA,M,F,...,18670,5415,3610,1,1,1,1,1,In-Store Coupon,95.78


# Model Development

In [5]:
y = df_raw['cost']
X = df_raw.drop(columns = ['cost'])
X = pd.get_dummies(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train_final, X_test_final, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [7]:
X_test_final, X_val_final, y_test, y_val = train_test_split(X_test_final, y_test, test_size = 0.5, random_state = 0)

# TPOT

In [8]:
! pip install tpot

In [9]:
from tpot import TPOTRegressor
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=0)
tpot_model = tpot.fit(X_train_final, y_train)
y_val_pred_tpot = tpot_model.predict(X_val_final)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -1.1228307693669495

Generation 2 - Current best internal CV score: -1.1228307693669495

Generation 3 - Current best internal CV score: -1.1228307693669495

Generation 4 - Current best internal CV score: -1.1228307693669495

Generation 5 - Current best internal CV score: -1.1228307693669495

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.25, min_samples_leaf=3, min_samples_split=4, n_estimators=100)


In [10]:
from sklearn.metrics import mean_squared_error
tpot_pred_mse = mean_squared_error(y_val, y_val_pred_tpot, squared=False)
tpot_pred_mse

0.9168477796296907